Loading *watermark* extension and printing details about running platform.

In [1]:
%load_ext watermark
%watermark -v -n -m -p numpy,pandas,sklearn,tensorflow,keras

Mon Apr 29 2019 

CPython 3.7.1
IPython 7.4.0

numpy 1.16.3
pandas 0.24.2
sklearn 0.20.3
tensorflow 1.13.1
keras 2.2.4

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-47-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


Using TensorFlow backend.


## Quick Exploration of Svebolle Deployment

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)

In [ ]:
DATASET_PATH = 'src/data/raw'

In [ ]:
data_fname = os.path.join(PROJ_ROOT, DATASET_PATH, 'LORA_data.csv')
dframe = pd.read_csv(data_fname, header=0, sep=';')
dframe.head()

In [ ]:
data = dframe.copy()

In [ ]:
data = data[['Time', 'DevAddr']]
data.head()

## Feature Engineering

In [ ]:
def remove_ns(data):
    Time = list(data.Time.values)
    Time_parsed = []
    sep = '.'
    for t in Time:
        t_parsed = t.split(sep, 1)[0]
        Time_parsed.append(t_parsed)
    data.Time = Time_parsed
    data['Time'] = pd.to_datetime(data['Time'])
    return data
        
dev1 = remove_ns(data)
dev1.head()

In [ ]:
dev1 = data[data.DevAddr == '000013c1']
dev1.head()

In [ ]:
dev1 = dev1.reset_index(drop=True)
dev1

In [ ]:
dev1 = dev1.Time

In [ ]:
dev1

In [ ]:
dev1_clean = dev1.drop_duplicates(keep='last')

In [ ]:
dev1_clean

In [ ]:
type(dev1_clean)

In [ ]:
df = pd.DataFrame({'Time':dev1_clean.values})

In [ ]:
df['Active'] = 1

In [ ]:
df = df.set_index('Time')

In [ ]:
df.index = pd.to_datetime(df.index)
df

In [ ]:
df_temp = df.resample('1S').asfreq()

In [ ]:
df_temp.fillna(0)

# LSTM

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras import backend as K
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=3, inter_op_parallelism_threads=3)))


In [ ]:
def timeDelay(data, delay):
    X_data, y_data = [], []
    
    for i in range(delay, len(data)):
        X_data.append(data[i-delay: i].tolist())
    X_data = np.array(X_data)
    y_data = data[delay:]
    return np.reshape(X_data, (X_data.shape[0], X_data.shape[1], 1)),\
           np.reshape(y_data, (len(y_data), ))

In [ ]:
def split(X, y, ratio):    
    test_split = int(len(X) * ratio)
    
    X_train, y_train = X[:test_split], y[:test_split]
    X_test, y_test = X[test_split:], y[test_split:]
    return X_train, y_train, X_test, y_test

In [ ]:
X, y = timeDelay(df_temp.Active.values, 5) # train test
ratio = 0.8 # train-test ratio
window_size = 5 # delay

X_train, y_train, X_test, y_test = split(X, y, ratio)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(window_size, 1))) # 50 neurons
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='adam')

In [ ]:
history = model.fit(X_train, y_train, 
                        epochs=20, 
                        batch_size=32, 
                        validation_data=(X_test, y_test), 
                        verbose=2, 
                        shuffle=False)

In [ ]:
# generate predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
import math

# Estimate model performance
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))